Strated this code cause it seemed there was some clock drift apparent even after aligning spike trains to state0 DIO (i.e. spike_times{i} = spike_times - state0_dio{i} + (alignEvent{i} - state0_bpod{i}). Drift appeard in the form of a peak with a constant time offset of a few ms over consecutive trials. That was observed before a bug with subtracting the first element of state0_dio from the entire vector, but not of the corresponding spike times - and hasn't been obvserved after, so I'm ignoring it for now.

In [4]:
import os
import re
import sys

import numpy as np
import matplotlib as mp
import pandas as pd
import scipy as sp
import scipy.stats as spt
import scipy.io as sio
import matplotlib.pyplot as plt

from tasks import dual2afc
from traum import traum as tr
#from traum.mdaio import readmda
#from traum import readDIO
#from traum.readTrodesExtractedDataFile import readTrodesExtractedDataFile as readdat

print(sys.version)

2.7.14 |Anaconda, Inc.| (default, Mar 27 2018, 12:28:59) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


## Behavior

In [5]:
subjName = 'M14'
sessName = 'M14_20171117_175330'

pathBhv = os.path.join('datasets/dual2afc_ds1/','bhv',subjName,'M14_Dual2AFC_Nov17_2017_Session1.mat')
dataBhv = dual2afc.parser(sio.loadmat(pathBhv, squeeze_me=True))

In [6]:
obj = tr.traum(dataBhv)

## Neural

In [7]:
pathNeur = os.path.join('datasets/dual2afc_ds1/','neur',subjName,sessName)

obj.readNeur(pathNeur)

## Sync

In [8]:
pathDIO = os.path.join('datasets/dual2afc_ds1/','neur',subjName,sessName,'dio')
obj.readDio(pathDIO)

obj.sync()

In [9]:
obj.neur

,cluster,dataset,spikes
0,1.0,ms3--t33,"[0.021866666666666666, 7.832633333333333, 7.90..."
1,2.0,ms3--t33,"[1.5329333333333333, 2.4887333333333332, 2.601..."
2,3.0,ms3--t33,"[1.6953, 3.0233333333333334, 3.459033333333333..."
3,9.0,ms3--t33,"[0.32306666666666667, 9.552166666666666, 12.84..."
4,10.0,ms3--t33,"[17.279366666666668, 23.888933333333334, 23.89..."
5,21.0,ms3--t33,"[0.34153333333333336, 0.5774666666666667, 0.59..."
6,27.0,ms3--t33,"[0.18603333333333333, 0.34186666666666665, 0.6..."
7,39.0,ms3--t33,"[0.0483, 0.076, 0.307, 0.5492, 1.0055666666666..."
8,1.0,ms3--t34,"[0.0746, 0.09333333333333334, 0.1247, 0.1494, ..."
9,14.0,ms3--t34,"[0.5899333333333333, 0.9094666666666666, 2.674..."


In [10]:
len(obj.neur)

32

## Traum

In [7]:
#ndxType = np.full(len(obj.bhv.parsedData),0)

corr_ndxType = np.full(len(obj.bhv.parsedData),0)
corr_ndxType[obj.bhv.parsedData['isCorrect']] = 1
corr_ndxType[obj.bhv.parsedData['isIncorr']] = 2
corr_colors = ['xkcd:grass green', 'xkcd:scarlet']

cho_ndxType = np.full(len(obj.bhv.parsedData),0)
cho_ndxType[obj.bhv.parsedData['isChoiceLeft']] = 1
cho_ndxType[obj.bhv.parsedData['isChoiceRight']] = 2
cho_colors =['xkcd:golden yellow', 'xkcd:dark sky blue']

stim_ndxType = np.full(len(obj.bhv.parsedData),0)
setStim = list(set(obj.bhv.parsedData['OdorFracA']))
for iStim in range(len(setStim)):
    stim_ndxType[obj.bhv.parsedData['OdorFracA'] == setStim[iStim]] = iStim+1
stim_colors = ['xkcd:bright blue', 'xkcd:deep sky blue', 'xkcd:bright sky blue', 'xkcd:aqua', 'xkcd:aquamarine', 'xkcd:shamrock green', 'xkcd:kermit green']

corr_trialMask = (corr_ndxType, corr_colors)
cho_trialMask = (cho_ndxType, cho_colors)
stim_trialMask = (stim_ndxType, stim_colors)

trialMask = (stim_trialMask,cho_trialMask,corr_trialMask)

In [8]:
listAlign = ('tsCin','tsStimOn','tsStimOff','tsChoice','tsRwd','tsErrTone')

In [ ]:
for iUnit in range(len(obj.neur)):
    figTitle = sessName + '_unit' + str(iUnit)
    hf, ha = plt.subplots(len(trialMask)*2,len(listAlign),figsize=(20,10))

    for iMask in range(len(trialMask)):
        for iAlign in range(len(listAlign)) :
            h = (ha[iMask*2,iAlign], ha[iMask*2+1,iAlign])# + '\t' + str() + str(iAlign) + '\n')
            obj.raspeth(listAlign[iAlign],iUnit,trialMask[iMask],h)

            if (iMask == 0) :
                ha[iMask*2,iAlign].set_title(listAlign[iAlign])

            if (iAlign == 0) :
                if (iMask == 0) :
                    ha[iMask*2,iAlign].set_ylabel('Trial #')
                    ha[iMask*2+1,iAlign].set_ylabel('Firing rate')
                elif (iMask == len(list(trialMask))) :
                    ha[iMask*2+1,iAlign].set_xlabel('Time (s)')

    #plt.suptitle(figTitle,fontsize=20)
    plt.tight_layout()
    plt.savefig(os.path.join('figs',figTitle + '_rice.pdf'))#,dpi=150,orientation='landscape',papertype='letter',format='eps')